This is a classification example to show how to use Oboe for training and testing, in the context of AutoML, i.e., do model selection on the training set and then evaluate the performance of the selected model on the test set.

In [1]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#import AutoLearner module by either specifying its relative path or doing pip installation
automl_path = '../automl'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Example 1: a no-brainer use

In [3]:
# initialize the autolearner class
m = AutoLearner(p_type='classification', runtime_limit=30, method='Oboe', verbose=False)

In [4]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [5]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.0
elapsed time: 29.56807851791382


In [6]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'select at most 5 pipelines with smallest cv error',
 'base learners': {'kSVM': [{'C': 4, 'kernel': 'rbf', 'coef0': 0},
   {'C': 4, 'kernel': 'rbf', 'coef0': 10},
   {'C': 16, 'kernel': 'rbf', 'coef0': 10},
   {'C': 8, 'kernel': 'rbf', 'coef0': 0},
   {'C': 4, 'kernel': 'rbf', 'coef0': 10}]}}

# Example 2: build an ensemble of models

In [7]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 30

In [8]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [9]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'ED',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
}

In [10]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [11]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [12]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))
print("individual accuracies of selected models: {}".format(m.get_model_accuracies(y_test)))

prediction error: 0.025438596491228094
elapsed time: 29.11527419090271
individual accuracies of selected models: [0.0, 0.0, 0.025, 0.025, 0.025]


In [13]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'select at most 5 pipelines with smallest cv error',
 'base learners': {'KNN': [{'n_neighbors': 5, 'p': 2},
   {'n_neighbors': 5, 'p': 2}],
  'ExtraTrees': [{'min_samples_split': 2, 'criterion': 'entropy'},
   {'min_samples_split': 0.01, 'criterion': 'entropy'},
   {'min_samples_split': 2, 'criterion': 'entropy'}]}}

# Example 3: just select a collection of promising models without building an ensemble afterwards

In [14]:
#experimental settings
VERBOSE = True #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [15]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [16]:
#autolearner arguments
autolearner_kwargs = {
    'method': 'Oboe',
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'ED',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': False,
}

In [17]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [18]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [19]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
 
print("elapsed time: {}".format(elapsed_time))
print("accuracies of selected models: {}".format(m.get_pipeline_accuracies(y_test)))

elapsed time: 14.332764387130737
accuracies of selected models: [0.025, 0.025, 0.025, 0.025, 0.0, 0.025, 0.025, 0.0, 0.0, 0.025, 0.025, 0.025, 0.051]


Note that we do not have a single accuracy value here if we do not build an ensemble, instead, we just have a collection of fitted models with individual accuracies reported.

In [20]:
# get names of the selected machine learning models
m.get_models()

{'selected models': {'KNN': [{'n_neighbors': 1, 'p': 1},
   {'n_neighbors': 1, 'p': 1},
   {'n_neighbors': 1, 'p': 2},
   {'n_neighbors': 3, 'p': 1},
   {'n_neighbors': 5, 'p': 1},
   {'n_neighbors': 3, 'p': 2},
   {'n_neighbors': 7, 'p': 1},
   {'n_neighbors': 5, 'p': 2},
   {'n_neighbors': 7, 'p': 2}],
  'DT': [{'min_samples_split': 1e-05},
   {'min_samples_split': 0.0001},
   {'min_samples_split': 2},
   {'min_samples_split': 32}]}}